<a href="https://colab.research.google.com/github/An0816/CGAN/blob/main/CGAN_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- Dropout 효과  
  : overfitting 방지  
  : 성능 향과    
  : 앙상블 효과( 매번 다른 형태의 노드로 학습하기 때문에)  

- One hot Encoding  
  : 데이터를 수많은 0과 한개의 1로 데이터를 구별하는 것
  : `scatter_(축, 새로 나타낼 인덱스, 새로 저장할 입력값)`  
  : `unsqueeze()` = 특정 위치에 1인 차원을 추가   

- `torch.randn()` : 평균이 0이고 std가 1인 가우시안 정규분포를 이용해서 생성   
- `torch.randint()` : 주어진 범위 내에서 정규를 균등하게 생성  
- torch.ones() : 주어진 사이즈의 1로 이루어진 텐서 생성   
- torch.zeros() : 주어진 사이즈의 0으로 이루어진 텐서 생성  




In [ ]:
import torch
import torch.nn as nn
import time
from torchvision import datasets, transforms
from torchvision.utils import save_image, make_grid
from torch.utils.data import DataLoader
from torch.autograd import Variable

In [ ]:
train_img = datasets.MNIST(root = 'MNIST/', train = True,
                           download = True, transform = transforms.Compose([
                                transforms.ToTensor(),
                                transforms.Normalize(mean = (0.5,), std = (0.5,))
                           ]))
batch_size = 100
train_data = DataLoader(dataset = train_img, batch_size = 100, shuffle = True, drop_last = True)

In [ ]:
# fake image를 생성해야 하므로, 마지막 layer에서 784개의 값을 만들어 내도록 해야 함
class Generator(nn.Module):
    def __init__(self):
        super().__init__()

        self.model = nn.Sequential(
            nn.Linear(110, 256), # 110 = noise + label
            nn.ReLU(),
            nn.Dropout(), # 노드를 무작위로 껐다 켰다 반복하는 것
            nn.Linear(256, 512),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(512, 1024),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(1024, 784),
            nn.Tanh()
        )
    def forward(self, x, y):
        x = torch.cat((x, y), dim = 1) # concatenation
        x = self.model(x) # forward propagation
        return x

# 어떤 데이터가 진짜인지 가짜인지를 판단해야 하므로 결과적으로 하나의 확률값을 만들어내야 함
# 확률값으로 나타내야 하기 때문에 마지막에 Sigmoid함수 사용
class Discriminator(nn.Module):
    def __init__(self):
        super().__init__()

        self.model = nn.Sequential(
            nn.Linear(794, 1024), # 이미지 하나가 784값을 가짐 + label(0~9 = 10개)
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(256, 1),
            nn.Sigmoid()
        )
    def forward(self, x, y):
        x = torch.cat((x, y), dim = 1)
        x = self.model(x)
        return x


In [ ]:
def one_hot_vector(labels, C = 10):
    one_hot = torch.FloatTensor(labels.size(0), C).zero_().to(device)
    target = one_hot.scatter_(1, labels.unsqueeze(1), 1) # scatter_(dim, index, src)
    target = Variable(target)
    return target

In [ ]:
batch_size = 100
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
G = Generator().to(device)
D = Discriminator().to(device)

optim_G = torch.optim.Adam(G.parameters(), lr = 0.001)
optim_D = torch.optim.Adam(D.parameters(), lr = 0.001)
criterion = torch.nn.BCELoss()

In [ ]:
# train
total_batch = len(train_data) # 600

for epoch in range(200):
    avg_cost = [0, 0]
    for x, y in train_data:
        x = x.view(x.size(0), -1).to(device)
        x_oh = one_hot_vector(y.to(device), 10)

        z = torch.randn(batch_size, 100, device = device) # noise (random하게 noise 생성)
        z_label = torch.randint(10, (batch_size,), device = device)
        z_oh = one_hot_vector(z_label, 10)
        fake_img = G(z, z_oh)

        real = (torch.FloatTensor(x.size(0), 1).fill_(1.0)).to(device)
        fake = (torch.FloatTensor(x.size(0), 1).fill_(0.0)).to(device)

        # train Generator
        optim_G.zero_grad()
        g_cost = criterion(D(fake_img, z_oh), real) 
        g_cost.backward()
        optim_G.step()

        fake_img = fake_img.detach().to(device)
        # train Discriminator
        optim_D.zero_grad()
        d_cost = criterion(D(torch.cat((x, z_img)), torch.cat((x_oh, z_oh))), torch.cat((real, fake)))
        d_cost.backward()
        optim_D.step()

        avg_cost[0] += g_cost
        avg_cost[1] += d_cost
    avg_cost[0] /= total_batch
    avg_cost[1] /= total_batch

    if (epoch+1) % 10 == 0 or epoch < 10:
        print(f"Epoch : {epoch + 1}, Generator : {avg_cost[0]}, Discriminator : {avg_cost[1]}")
        z = torch.rand(100, 100, device = device)
        label = torch.Tensor(100).fill_(0).long().to(device)
        for i in range(10):
            for j in range(10):
                label[10*i+j] = j
        z_oh = one_hot_vector(label, 10)
        fake_img = G(z, z_oh)
        fake_img = fake_img.reshape([100, 1, 28, 28])
        img_grid = make_grid(fake_img, nrow = 10, normalize = True)
        save_image(img_grid, "/content/drive/MyDrive/Deep Learning/GAN/GAN Result/%d.png"%(epoch+1))
